In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import math

In [60]:
def convert_state_to_list(state):
    return [int(x) for x in str(state)]

def convert_list_to_state(state_list):
    return sum([s*10**(len(state_list)-1-j) for j, s in enumerate(state_list)])

In [61]:
sl = convert_state_to_list(313212313)
print(sl)
ls = convert_list_to_state(sl)
print(ls)

[3, 1, 3, 2, 1, 2, 3, 1, 3]
313212313


In [62]:
def check_for_win(state, x, o):
    state_list = [int(x) for x in str(state)]
    
    # check columns
    for i in range(3):
        val = state_list[0+i]*state_list[3+i]*state_list[6+i]
        if val == x**3 or val == o**3:
            return int(val**(1/3))
        
    # check rows
    for i in range(3):
        val = state_list[0+(i*3)]*state_list[1+(i*3)]*state_list[2+(i*3)]
        if val == x**3 or val == o**3:
            return int(val**(1/3))
    
    # check diagonals
    for i in range(2):
        val = state_list[0+(2*i)]*state_list[4]*state_list[8-(2*i)]
        if val == x**3 or val == o**3:
            return int(val**(1/3))
        
    return 0    # represents that no one has won yet


def game_over(state, x, o):
    winner = check_for_win(state, x, o)
    if winner:
        return winner
    elif not "1" in str(state):
        return 0    # represents a draw
    else:
        return -1   # represents that the game is not over yet

In [63]:
#def init_state_values():
#    state_value_dict = {}
#    for state in tqdm(range(111111111, 333333333 + 1)):
#        check = True
#        for i in [0,4,5,6,7,8,9]:
#            if str(i) in str(state):
#                check = False
#                break
#        if check == True:
#            state_value_dict[state] = 0.5
#            
#    return state_value_dict

In [64]:
def get_next_states(current_state, player):
    state_list = [int(x) for x in str(current_state)]

    next_states_list = []
    for i in range(len(state_list)):
        if state_list[i] == 1:
            possible_state_list = state_list[:] # copy list by value (hopefully!)
            possible_state_list[i] = player
            possible_state = convert_list_to_state(possible_state_list)
            next_states_list.append(possible_state)

    return next_states_list

def select_next_state(state_value_dict, next_state_list, selection_criterion):
    value_list = []
    for next_state in next_state_list:
        if next_state in state_value_dict.keys():
            state_value = state_value_dict[next_state]
            value_list.append(state_value)
        else:
            value_list.append(0.5)
    next_state = random.choices(population=next_state_list, weights=value_list, k=1)[0]
    if not next_state in state_value_dict.keys():
        state_value_dict[next_state] = 0.5

    return next_state, state_value_dict

def take_action(state_value_dict, current_state, player, selection_criterion):
    next_state_list = get_next_states(current_state, player)
    next_state, state_value_dict = select_next_state(state_value_dict, next_state_list, selection_criterion)

    return next_state, state_value_dict

In [65]:
get_next_states(current_state=213112111, player=2)

[223112111, 213212111, 213122111, 213112211, 213112121, 213112112]

In [66]:
state_value_dict = {} #init_state_values()

In [67]:
len(state_value_dict.keys()) == 3**9

False

In [103]:
game_over(233233222, 2, 3)

2

In [2]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [151]:
softmax([100,2,3])

array([1.00000000e+00, 2.74878501e-43, 7.47197234e-43])

In [255]:
class TTTAgent():
    def __init__(self, step_size, default_value, exploration_discounting, player_description, strategy) -> None:
        self.state_value_dict = {}
        self.step_size = step_size
        self.default_value = default_value
        self.exp_disc = exploration_discounting
        self.strategy = strategy
        if player_description == "X":
            self.player_description = 2
        elif player_description == "O":
            self.player_description = 3
        else:
            print("ERROR, invalid, player description")     # replace with actual error

        self.experience = 1     # gets updated by one after every game played, or won?
        self.action_list = []

    def convert_list_to_state(self, state_list):
        return sum([s*10**(len(state_list)-1-j) for j, s in enumerate(state_list)])

    def _get_next_states(self, current_state):
        state_list = [int(x) for x in str(current_state)]

        next_states_list = []
        for i in range(len(state_list)):
            if state_list[i] == 1:
                possible_state_list = state_list[:] # copy list by value (hopefully!)
                possible_state_list[i] = self.player_description
                possible_state = self.convert_list_to_state(possible_state_list)
                next_states_list.append(possible_state)

        return next_states_list

    def _select_next_state(self, next_state_list):
        value_list = []
        optimal_action = False

        for next_state in next_state_list:
            if next_state in self.state_value_dict.keys():
                state_value = self.state_value_dict[next_state]
                value_list.append(state_value)
            else:
                value_list.append(self.default_value)

        if self.strategy == "exploratory":
            next_state = random.choices(
                population=next_state_list, 
                weights=softmax([value*(10-math.log(self.exp_disc**math.sqrt(self.experience))) for value in value_list]), 
                k=1
            )[0]        # that's the policy
        else:
            next_state = next_state_list[np.argmax(value_list)] 
            
        if not next_state in self.state_value_dict.keys():
            self.state_value_dict[next_state] = self.default_value

        if self.state_value_dict[next_state] == max(value_list):
            optimal_action = True

        return next_state, optimal_action

    def take_action(self, current_state):
        next_state_list = self._get_next_states(current_state)
        next_state, optimal_action = self._select_next_state(next_state_list)

        self.action_list.append((next_state, optimal_action))

        return next_state, optimal_action
    
    def _update_value(self, state_t, state_t1):
        value_t = self.state_value_dict[state_t]
        value_t1 = self.state_value_dict[state_t1]

        self.state_value_dict[state_t] += self.step_size*(value_t1-value_t)#**math.sqrt(self.experience)))

    def update_values(self, reward):

        #print(self.action_list)
        
        last_state = self.action_list[-1][0]
        self.state_value_dict[last_state] = reward

        for i in range(len(self.action_list)-1, 1, -1):
            optimal_move = self.action_list[i][1]

            if optimal_move:
                state_prev = self.action_list[i-1][0]
                state_now = self.action_list[i][0]

                self._update_value(state_prev, state_now)

        self.experience += 0.1
        self.action_list = []



In [256]:
class TTTEnv():
    def __init__(self) -> None:
        self.state_list = []
        self.player_x = 2
        self.player_o = 3
        self.winner = None

    def move(self, player, action):
        self.state_list.append((action, player))

        return self.game_over(action, player)

    def check_for_win(self, state, player):
        state_list = [int(x) for x in str(state)]
        
        # check columns
        for i in range(3):
            val = state_list[0+i]*state_list[3+i]*state_list[6+i]
            if val == player**3:
                return 1
            
        # check rows
        for i in range(3):
            val = state_list[0+(i*3)]*state_list[1+(i*3)]*state_list[2+(i*3)]
            if val == player**3:
                return 1
        
        # check diagonals
        for i in range(2):
            val = state_list[0+(2*i)]*state_list[4]*state_list[8-(2*i)]
            if val == player**3:
                return 1
            
        return -1    # represents that no one has won yet


    def game_over(self, state, player):
        reward = self.check_for_win(state, player)
        if reward > 0:
            self.winner = player
            return reward
        elif not "1" in str(state):
            return -1    # represents a draw
        else:
            return 0   # represents that the game is not over yet

In [275]:
agent_1 = TTTAgent(
    step_size=0.9,
    default_value=0.5,
    exploration_discounting=0.9,
    player_description="X",
    strategy="greedy"
)

agent_2 = TTTAgent(
    step_size=0.9,
    default_value=0.5,
    exploration_discounting=0.9,
    player_description="O",
    strategy="exploratory"
)


env = TTTEnv()

In [276]:

players = [agent_1, agent_2]

num_epochs = 1000000
win_dict = {
    2: 0,
    3: 0
}

for epoch in tqdm(range(num_epochs)):
    current_state = 111111111
    game_finished = False
    players.reverse()
    while not game_finished:
        for i in range(len(players)):
            next_state, _ = players[i].take_action(current_state)
            current_state = next_state
            reward = env.game_over(current_state, players[i].player_description)

            if reward == 1:
                win_dict[env.winner] += 1
                players[i].update_values(reward)
                players[i-1].update_values(-reward)
                game_finished = True
                break
            elif reward == -1:
                players[i].update_values(0)
                players[i-1].update_values(0)
                game_finished = True
                break


100%|██████████| 1000000/1000000 [04:47<00:00, 3484.00it/s]


In [277]:
win_dict

{2: 312659, 3: 222911}

In [278]:
len(agent_1.state_value_dict.keys())

1122

In [279]:
len(agent_2.state_value_dict.keys())

1058

In [ ]:
# build interface, to play against agent